##### Copyright 2019 The IREE Authors

In [1]:
#@title Licensed under the Apache License v2.0 with LLVM Exceptions.
# See https://llvm.org/LICENSE.txt for license information.
# SPDX-License-Identifier: Apache-2.0 WITH LLVM-exception

# Low Level Invoke Function

This notebook shows off some concepts of the low level IREE python bindings.

In [2]:
!python -m pip install iree-compiler-snapshot iree-runtime-snapshot -f https://github.com/google/iree/releases

Looking in links: https://github.com/google/iree/releases
     |████████████████████████████████| 33.6MB 118kB/s 
     |████████████████████████████████| 522kB 27.1MB/s 


In [3]:
import numpy as np

from iree import runtime as ireert
from iree.compiler import compile_str

In [7]:
# Compile a module.
SIMPLE_MUL_ASM = """
  module @arithmetic {
    func @simple_mul(%arg0: tensor<4xf32>, %arg1: tensor<4xf32>) -> tensor<4xf32>
          {
        %0 = mulf %arg0, %arg1 : tensor<4xf32>
        return %0 : tensor<4xf32>
    } 
  }
"""

compiled_flatbuffer = compile_str(SIMPLE_MUL_ASM, target_backends=["vmvx"])
vm_module = ireert.VmModule.from_flatbuffer(compiled_flatbuffer)

In [8]:
# Register the module with a runtime context.
# Use the CPU interpreter (which has the most implementation done):
config = ireert.Config("vmvx")
ctx = ireert.SystemContext(config=config)
ctx.add_vm_module(vm_module)

# Invoke the function and print the result.
print("INVOKE simple_mul")
arg0 = np.array([1., 2., 3., 4.], dtype=np.float32)
arg1 = np.array([4., 5., 6., 7.], dtype=np.float32)
f = ctx.modules.arithmetic["simple_mul"]
results = f(arg0, arg1)
print("Results:", results)

INVOKE simple_mul
Results: [ 4. 10. 18. 28.]


Created IREE driver vmvx: <iree.runtime.binding.HalDriver object at 0x7f9b9d6e6ef0>
SystemContext driver=<iree.runtime.binding.HalDriver object at 0x7f9b9d6e6ef0>
